# Detect the bottom dots on an image and use it to construct a spatial mapping

## Read, open and display the image

In [ ]:
# Load modules
import cv2
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['figure.dpi'] = 200

# Read image
path = './../data/calibration/exampledots.tiff'

bitdepth = 16

# If image has bitdepth 8 or it has sharp contrasts making grayscale values obsolete
if bitdepth == 8:
    image = cv2.imread(path,cv2.CV_8UC1) * 16
# 
elif bitdepth == 16:
    image = cv2.imread(path,cv2.CV_16UC1)

# Show image
plt.figure()
plt.imshow(image*1,origin='lower',cmap='gray')
plt.show()

## Extract grid data from loaded image

In [ ]:
# Fixed thresholding
meanvalue = int(np.mean(image))
thresholdvalue = meanvalue/1.5
binaryImage = cv2.threshold(image,thresholdvalue,255,cv2.THRESH_BINARY_INV)[1]

# Define a course kernel
kernelClosed = np.ones((7,7),np.uint8)
kernelOpen   = np.ones((5,5),np.uint8)

# Reduce noise inside particles
binaryImageClosed = cv2.morphologyEx(binaryImage, cv2.MORPH_CLOSE, kernelClosed)

# Reduce noise outside particles
binaryImageOpened = cv2.morphologyEx(binaryImageClosed, cv2.MORPH_OPEN, kernelOpen)

markers = cv2.connectedComponents(np.uint8(binaryImageOpened))[1]

Nmarkers = len(np.unique(markers))
Ndots = int(13*7)
maximumSize = 50
minimumSize = 5

xdots = np.empty(0,dtype=float)
ydots = np.empty(0,dtype=float)

for i in range(1,Nmarkers,1):
    idx = np.argwhere(markers==i)
    
    xSpread = np.max(idx[:,1])-np.min(idx[:,1])    
    ySpread = np.max(idx[:,0])-np.min(idx[:,0])
    
    if ((minimumSize < xSpread < maximumSize) and (minimumSize < ySpread < maximumSize)):
        xdots = np.append(xdots,np.mean(idx[:,1]))
        ydots = np.append(ydots,np.mean(idx[:,0]))
    
plt.figure()
plt.imshow(image,origin='lower',cmap='gray')
plt.plot(xdots,ydots,'r.',markersize=1)
plt.show()

## Couple image to a spatial grid

In [ ]:
# Sort the dots
NRows = 7
NColumns = 13

for i in range(0,NRows,1):
    idx = np.argsort(xdots[i*NColumns:(i+1)*NColumns])
    
    xdots[i*NColumns:(i+1)*NColumns] = xdots[i*NColumns+idx]
    ydots[i*NColumns:(i+1)*NColumns] = ydots[i*NColumns+idx]


    
# Positions of the dots in meters
distanceDots = 0.02 # meters
x = np.linspace(0,(NColumns-1)*distanceDots,NColumns)
y = np.linspace(0,(NRows-1)   *distanceDots,NRows)
xReal, yReal = np.meshgrid(x,y)

# We need these two arrays as input data sets for the least square fititng
Bx = xReal.flatten()
By = yReal.flatten()

# Positions of the dots in pixels
X = xdots*1
Y = ydots*1


A = np.array([X*0+1, X, Y, X**2, X**2*Y, X**2*Y**2, Y**2, X*Y**2, X*Y]).T

# Use linear least square fitting to find the coefficients that solve A*x=B
coeffx, _, _, _ = np.linalg.lstsq(A,Bx,rcond=None)
coeffy, _, _, _ = np.linalg.lstsq(A,By,rcond=None)


# This is the function we just used for fitting, but here explicit
def surface(x,y, a,b,c,d,e,f,g,h,i):
    return a + b*x + c*y + d*x**2 + e*x**2*y + f*x**2*y**2 + g*y**2 + h*x*y**1 + i*x*y


# Plot and compare datapoints with fitted surfaces
xplot,yplot = np.meshgrid(np.linspace(0,1500,100),np.linspace(0,1500,100))

xplot = xplot.flatten()
yplot = yplot.flatten()

fig = plt.figure()
ax = fig.add_subplot(111,projection='3d')
ax.scatter(X,Y,xReal)
ax.plot_trisurf(xplot,yplot,surface(xplot,yplot,*coeffx),alpha=0.3)
ax.set_xlabel('x img [pixels]')
ax.set_ylabel('y img [pixels]')
ax.set_zlabel('x real [m]')
plt.show()

fig = plt.figure()
ax = fig.add_subplot(111,projection='3d')
ax.scatter(X,Y,yReal)
ax.plot_trisurf(xplot,yplot,surface(xplot,yplot,*coeffy),alpha=0.3)
ax.set_xlabel('x img [pixels]')
ax.set_ylabel('y img [pixels]')
ax.set_zlabel('y real [m]')
plt.show()